# K7 GIFT v1.7 - Hybrid Analytic-ML Construction

**Goal**: Upgrade from v1.6 by integrating an **analytic backbone for phi** and letting the neural network learn only the **residual delta_phi**.

## Key Features:
- **Analytic backbone** `phi_back(lambda)` for dominant components (phi_012, phi_013)
- **Residual network** `delta_phi_net` learns corrections only
- **Enriched analytic basis** for metric/phi extraction
- **Optional symbolic regression** for compact formulas

## Targets (from v1.6):
- kappa_T = 1/61 (torsion)
- det(g) = 65/32 (metric determinant)
- b2 = 21, b3 = 77 (Betti numbers)

## Previous Results (v1.6 analytical ansatz):
- phi_012(lambda): R^2 = 0.85
- phi_013(lambda): R^2 = 0.81

## 1. Imports and Configuration

In [ ]:
"""
K7 GIFT v1.7 - Hybrid Analytic-ML Construction
==============================================
Imports and setup
"""
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional, List
import json
import math
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set device and precision
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# =============================================================================
# Configuration v1.7
# =============================================================================

CONFIG = {
    'version': '1.7',
    
    # Paths
    'v16_model_path': '../1_6/data/models_v1_6.pt',
    'v16_coords_path': '../1_6/sample_coords_v1_6.pt',
    'v16_results_path': '../1_6/data/results_v1_6.json',
    
    # Physical targets
    'kappa_T_target': 1.0 / 61.0,       # Torsion constant
    'det_g_target': 65.0 / 32.0,        # Metric determinant
    'b2_target': 21,                     # Second Betti number
    'b3_target': 77,                     # Third Betti number (35 local + 42 global)
    
    # Backbone mode: 'full' or 'compressed'
    'backbone_mode': 'full',
    
    # Residual network architecture
    'residual_net': {
        'hidden_dims': [64, 64, 32],
        'fourier_features': 16,
        'activation': 'silu',
    },
    
    # Training parameters
    'n_points': 2048,
    'n_epochs': 1500,
    'lr_residual': 5e-4,
    'weight_decay': 1e-6,
    
    # Loss weights
    'loss_weights': {
        'kappa_T': 200.0,
        'det_g': 5.0,
        'closure': 1.0,
        'coclosure': 1.0,
        'g2_consistency': 2.0,
        'residual_norm': 100.0,     # Penalize large residuals
        'spd': 5.0,
    },
    
    # Training phases
    'phases': [
        {'name': 'warmup', 'epochs': 300, 'residual_weight': 1.0, 'lr_factor': 1.0},
        {'name': 'relax', 'epochs': 700, 'residual_weight': 0.3, 'lr_factor': 0.5},
        {'name': 'finetune', 'epochs': 500, 'residual_weight': 0.1, 'lr_factor': 0.1},
    ],
    
    # TCS parameters (from v1.6)
    'tcs': {
        'neck_half_length': 1.0,
        'neck_width': 0.3,
        'twist_angle': math.pi / 4,
        'left_scale': 1.0,
        'right_scale': 1.0,
    },
    
    # Enriched basis for extraction
    'enriched_basis': {
        'max_polynomial_order': 3,
        'max_fourier_k': 4,
        'include_mixed_terms': True,
        'include_bump_functions': True,
    },
    
    # Optional symbolic regression
    'use_symbolic_regression': False,
}

print("Configuration loaded:")
print(f"  Backbone mode: {CONFIG['backbone_mode']}")
print(f"  Residual net: {CONFIG['residual_net']['hidden_dims']}")
print(f"  Training epochs: {CONFIG['n_epochs']}")

## 2. Structural Constants and Zero-Parameter Geometry

These are the fundamental constants derived from E8 x E8 structure:

In [ ]:
# =============================================================================
# Structural Constants (Zero-Parameter Framework)
# =============================================================================

class StructuralConstants:
    """
    All constants derived from E8 x E8 exceptional structure.
    No free parameters - everything follows from geometry.
    """
    # E8 structure
    dim_E8 = 248
    rank_E8 = 8
    dim_E8xE8 = 496
    
    # G2 holonomy manifold K7
    dim_K7 = 7
    dim_G2 = 14
    
    # Betti numbers (topological)
    b2 = 21       # Harmonic 2-forms
    b3_local = 35  # Local 3-forms (T^7 structure)
    b3_global = 42 # Global 3-forms (TCS gluing)
    b3 = 77       # Total harmonic 3-forms
    
    # Representation dimensions
    n1 = 2   # Singlet scalars
    n7 = 21  # 7-rep modes = b2
    n27 = 54 # 27-rep modes
    
    # Key ratios
    p2 = dim_G2 / dim_K7  # = 2 (binary duality)
    H_star = b2 + b3 + 1  # = 99 (effective cohomological dimension)
    
    # Angular quantization
    beta_0 = math.pi / rank_E8  # = pi/8
    
    # Weyl factor (from |W(E8)| = 2^14 * 3^5 * 5^2 * 7)
    Weyl_factor = 5
    
    # Derived correlation parameter
    xi = (Weyl_factor / p2) * beta_0  # = 5*pi/16
    
    # Hierarchy parameter
    tau = (dim_E8xE8 * b2) / (n27 / 2 * H_star)  # = 496*21/(27*99)
    
    # Metric determinant target
    det_g_target = 65.0 / 32.0  # = 2.03125
    
    # Torsion target
    kappa_T_target = 1.0 / 61.0

SC = StructuralConstants()

print("Structural Constants:")
print(f"  E8 x E8 dimension: {SC.dim_E8xE8}")
print(f"  K7 dimension: {SC.dim_K7}")
print(f"  Betti numbers: b2={SC.b2}, b3={SC.b3} (={SC.b3_local}+{SC.b3_global})")
print(f"  Binary duality p2: {SC.p2}")
print(f"  xi = 5*pi/16 = {SC.xi:.6f}")
print(f"  tau = {SC.tau:.6f}")
print(f"  det(g) target: {SC.det_g_target}")
print(f"  kappa_T target: {SC.kappa_T_target:.6f}")

## 3. Load v1.6 Model and Sample Coordinates

Load the trained v1.6 model to initialize geometry components:

In [ ]:
# =============================================================================
# Load v1.6 artifacts
# =============================================================================

def load_v16_data():
    """Load v1.6 model, coordinates, and results."""
    base_path = Path(CONFIG['v16_model_path']).parent.parent
    
    # Load sample coordinates
    coords_path = base_path / 'sample_coords_v1_6.pt'
    if coords_path.exists():
        sample_coords = torch.load(coords_path, map_location=device)
        print(f"Loaded sample coordinates: {sample_coords.shape}")
    else:
        # Generate new coordinates if not found
        n_pts = CONFIG['n_points']
        sample_coords = torch.rand(n_pts, SC.dim_K7, device=device, dtype=torch.float64)
        print(f"Generated new sample coordinates: {sample_coords.shape}")
    
    # Load v1.6 results for reference
    results_path = Path(CONFIG['v16_results_path'])
    if results_path.exists():
        with open(results_path, 'r') as f:
            v16_results = json.load(f)
        print(f"Loaded v1.6 results: kappa_T={v16_results['achieved']['kappa_T']:.6f}")
    else:
        v16_results = None
        print("Warning: v1.6 results not found")
    
    return sample_coords, v16_results

sample_coords, v16_results = load_v16_data()

## 4. Analytic Backbone phi_back(lambda)

Define the analytical backbone for the dominant 3-form components based on v1.6 fitted ansatz.

The neck coordinate lambda = 2*x0 - 1 maps [0,1] -> [-1,1].

### Fitted formulas from v1.6:
- **phi_012**: R^2 = 0.85
- **phi_013**: R^2 = 0.81

In [ ]:
# =============================================================================
# Analytic Backbone phi_back(lambda)
# =============================================================================

# Fitted coefficients from v1.6 analysis
PHI_012_COEFFS = {
    'constant': 1.7052,
    'linear': -0.5459,
    'quadratic': -0.2684,
    'sin_pi': -0.4766,
    'cos_pi': -0.3704,
    'sin_2pi': -0.3303,
    'cos_2pi': -0.0992,
}

PHI_013_COEFFS = {
    'constant': 2.0223,
    'linear': 0.3633,
    'quadratic': -4.1523,
    'sin_pi': 0.1689,
    'cos_pi': -1.1874,
    'sin_2pi': -0.0514,
    'cos_2pi': 0.8497,
}

# Compressed coefficients (dominant terms only)
PHI_012_COMPRESSED = {
    'constant': 1.7,
    'linear': -0.5,
    'quadratic': -0.3,
}

PHI_013_COMPRESSED = {
    'constant': 2.0,
    'quadratic': -4.1,
    'cos_pi': -1.2,
}


def phi_back_012(lam: torch.Tensor, mode: str = 'full') -> torch.Tensor:
    """
    Analytic backbone for phi_012 component.
    
    Args:
        lam: Neck coordinate in [-1, 1], shape [N]
        mode: 'full' or 'compressed'
    
    Returns:
        phi_012 values, shape [N]
    """
    if mode == 'full':
        c = PHI_012_COEFFS
        return (c['constant']
                + c['linear'] * lam
                + c['quadratic'] * lam**2
                + c['sin_pi'] * torch.sin(math.pi * lam)
                + c['cos_pi'] * torch.cos(math.pi * lam)
                + c['sin_2pi'] * torch.sin(2.0 * math.pi * lam)
                + c['cos_2pi'] * torch.cos(2.0 * math.pi * lam))
    else:
        c = PHI_012_COMPRESSED
        return (c['constant']
                + c['linear'] * lam
                + c['quadratic'] * lam**2)


def phi_back_013(lam: torch.Tensor, mode: str = 'full') -> torch.Tensor:
    """
    Analytic backbone for phi_013 component.
    
    Args:
        lam: Neck coordinate in [-1, 1], shape [N]
        mode: 'full' or 'compressed'
    
    Returns:
        phi_013 values, shape [N]
    """
    if mode == 'full':
        c = PHI_013_COEFFS
        return (c['constant']
                + c['linear'] * lam
                + c['quadratic'] * lam**2
                + c['sin_pi'] * torch.sin(math.pi * lam)
                + c['cos_pi'] * torch.cos(math.pi * lam)
                + c['sin_2pi'] * torch.sin(2.0 * math.pi * lam)
                + c['cos_2pi'] * torch.cos(2.0 * math.pi * lam))
    else:
        c = PHI_013_COMPRESSED
        return (c['constant']
                + c['quadratic'] * lam**2
                + c['cos_pi'] * torch.cos(math.pi * lam))


# Test backbone
test_lam = torch.linspace(-1, 1, 100, device=device)
test_012 = phi_back_012(test_lam, mode='full')
test_013 = phi_back_013(test_lam, mode='full')

print(f"phi_012 backbone: min={test_012.min():.4f}, max={test_012.max():.4f}, mean={test_012.mean():.4f}")
print(f"phi_013 backbone: min={test_013.min():.4f}, max={test_013.max():.4f}, mean={test_013.mean():.4f}")

In [ ]:
class AnalyticBackbone(nn.Module):
    """
    Analytic backbone for the G2 3-form.
    
    Provides phi_back(x) with known analytical structure from v1.6 fits.
    The backbone captures ~85% of the variance in dominant components.
    """
    
    def __init__(self, mode: str = 'full'):
        super().__init__()
        self.mode = mode
        self.dim = 7
        
        # Standard G2 structure constants (for non-dominant components)
        # phi_G2 = dx^012 + dx^034 + dx^056 + dx^135 - dx^146 - dx^236 - dx^245
        self.g2_indices = [
            (0, 1, 2, +1.0),
            (0, 3, 4, +1.0),
            (0, 5, 6, +1.0),
            (1, 3, 5, +1.0),
            (1, 4, 6, -1.0),
            (2, 3, 6, -1.0),
            (2, 4, 5, -1.0),
        ]
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Compute analytic backbone phi_back.
        
        Args:
            x: Coordinates in [0, 1]^7, shape [N, 7]
        
        Returns:
            phi_back: 3-form tensor, shape [N, 7, 7, 7]
        """
        N = x.shape[0]
        phi = torch.zeros(N, 7, 7, 7, device=x.device, dtype=x.dtype)
        
        # Neck coordinate: lambda = 2*x0 - 1
        lam = 2.0 * x[:, 0] - 1.0
        
        # Dominant components from analytical fit
        phi_012_val = phi_back_012(lam, self.mode)
        phi_013_val = phi_back_013(lam, self.mode)
        
        # Fill phi_012 with antisymmetry
        phi[:, 0, 1, 2] = phi_012_val
        phi[:, 0, 2, 1] = -phi_012_val
        phi[:, 1, 0, 2] = -phi_012_val
        phi[:, 1, 2, 0] = phi_012_val
        phi[:, 2, 0, 1] = phi_012_val
        phi[:, 2, 1, 0] = -phi_012_val
        
        # Fill phi_013 with antisymmetry
        phi[:, 0, 1, 3] = phi_013_val
        phi[:, 0, 3, 1] = -phi_013_val
        phi[:, 1, 0, 3] = -phi_013_val
        phi[:, 1, 3, 0] = phi_013_val
        phi[:, 3, 0, 1] = phi_013_val
        phi[:, 3, 1, 0] = -phi_013_val
        
        # Other G2 structure components (constant baseline)
        # Scale to match typical magnitudes
        baseline_scale = 1.0
        for (i, j, k, sign) in self.g2_indices:
            if (i, j, k) not in [(0, 1, 2), (0, 1, 3)]:
                phi[:, i, j, k] = sign * baseline_scale
                phi[:, i, k, j] = -sign * baseline_scale
                phi[:, j, i, k] = -sign * baseline_scale
                phi[:, j, k, i] = sign * baseline_scale
                phi[:, k, i, j] = sign * baseline_scale
                phi[:, k, j, i] = -sign * baseline_scale
        
        return phi
    
    def get_norm(self, x: torch.Tensor) -> torch.Tensor:
        """Compute L2 norm of backbone."""
        phi = self.forward(x)
        return torch.sqrt((phi ** 2).sum(dim=(1, 2, 3))).mean()


# Test backbone
backbone = AnalyticBackbone(mode=CONFIG['backbone_mode']).to(device)
test_x = torch.rand(100, 7, device=device, dtype=torch.float64)
phi_back = backbone(test_x)

print(f"Backbone output shape: {phi_back.shape}")
print(f"Backbone norm: {backbone.get_norm(test_x):.4f}")

## 5. Residual Neural Network delta_phi_net

The residual network learns only corrections to the analytic backbone.
Much smaller than the full phi network since backbone handles ~85% of variance.

In [ ]:
# =============================================================================
# Fourier Feature Encoder (for residual network)
# =============================================================================

class FourierFeatures(nn.Module):
    """Random Fourier features for positional encoding."""
    
    def __init__(self, input_dim: int, n_features: int, scale: float = 1.0):
        super().__init__()
        B = torch.randn(input_dim, n_features) * scale
        self.register_buffer('B', B)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        xB = torch.matmul(x, self.B)
        return torch.cat([torch.sin(xB), torch.cos(xB)], dim=-1)


# =============================================================================
# Residual Phi Network
# =============================================================================

class ResidualPhiNet(nn.Module):
    """
    Neural network that outputs residual delta_phi.
    
    Learns only corrections to the analytic backbone.
    Output is antisymmetrized to ensure valid 3-form structure.
    """
    
    def __init__(self, config: dict):
        super().__init__()
        net_cfg = config['residual_net']
        hidden_dims = net_cfg['hidden_dims']
        n_fourier = net_cfg['fourier_features']
        
        # Fourier encoding
        self.fourier = FourierFeatures(7, n_fourier, scale=2.0)
        input_dim = 2 * n_fourier
        
        # Build MLP
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.SiLU())
            prev_dim = h_dim
        
        self.backbone = nn.Sequential(*layers)
        
        # Output head: produces raw (7,7,7) tensor coefficients
        # We output 35 independent components (7 choose 3)
        # then expand to full antisymmetric tensor
        self.n_components = 35
        self.head = nn.Linear(prev_dim, self.n_components)
        
        # Precompute index mapping for (i,j,k) with i<j<k
        self.indices = []
        idx = 0
        for i in range(7):
            for j in range(i + 1, 7):
                for k in range(j + 1, 7):
                    self.indices.append((i, j, k))
                    idx += 1
        
        # Initialize with small weights for small residuals
        nn.init.normal_(self.head.weight, std=0.01)
        nn.init.zeros_(self.head.bias)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Compute delta_phi (residual correction).
        
        Args:
            x: Coordinates in [0, 1]^7, shape [N, 7]
        
        Returns:
            delta_phi: Antisymmetric 3-form, shape [N, 7, 7, 7]
        """
        N = x.shape[0]
        
        # Fourier features
        h = self.fourier(x)
        
        # MLP backbone
        h = self.backbone(h)
        
        # Output 35 independent components
        coeffs = self.head(h)  # [N, 35]
        
        # Expand to antisymmetric tensor
        delta_phi = torch.zeros(N, 7, 7, 7, device=x.device, dtype=x.dtype)
        
        for idx, (i, j, k) in enumerate(self.indices):
            val = coeffs[:, idx]
            # Fill all 6 permutations with correct signs
            delta_phi[:, i, j, k] = val
            delta_phi[:, i, k, j] = -val
            delta_phi[:, j, i, k] = -val
            delta_phi[:, j, k, i] = val
            delta_phi[:, k, i, j] = val
            delta_phi[:, k, j, i] = -val
        
        # Scale down residual to prevent dominating backbone
        delta_phi = delta_phi * 0.3
        return delta_phi
    
    def get_norm(self, x: torch.Tensor) -> torch.Tensor:
        """Compute L2 norm of residual."""
        delta_phi = self.forward(x)
        return torch.sqrt((delta_phi ** 2).sum(dim=(1, 2, 3))).mean()


# Test residual network
residual_net = ResidualPhiNet(CONFIG).to(device)
delta_phi = residual_net(test_x)

print(f"Residual network output shape: {delta_phi.shape}")
print(f"Initial residual norm: {residual_net.get_norm(test_x):.6f}")
print(f"Number of parameters: {sum(p.numel() for p in residual_net.parameters())}")

## 6. Hybrid Model: phi_total = phi_back + delta_phi

Combine the analytic backbone with the learned residual, then compute metric and geometric quantities.

In [ ]:
# =============================================================================
# G2 Metric from 3-form
# =============================================================================

def phi_to_metric(phi: torch.Tensor) -> torch.Tensor:
    """
    Compute metric from G2 3-form using standard formula.
    
    g_ab = (1/144) * phi_{acd} * phi_{bef} * phi_{cdf}^{-1} * eps^{cdefgh}
    
    For practical computation, we use the simpler contraction formula
    valid for G2 structures.
    
    Args:
        phi: 3-form tensor, shape [N, 7, 7, 7]
    
    Returns:
        g: Metric tensor, shape [N, 7, 7]
    """
    N = phi.shape[0]
    
    # Compute g_ab = (1/6) sum_{cde} phi_{acd} phi_{bcd}
    # This is a simplified version - works for normalized G2 structures
    g = torch.einsum('nacd,nbcd->nab', phi, phi) / 6.0
    
    return g


def enforce_spd(g: torch.Tensor, eps: float = 1e-4) -> torch.Tensor:
    """
    Enforce symmetric positive definite metric.
    
    Args:
        g: Metric tensor, shape [N, 7, 7]
        eps: Minimum eigenvalue
    
    Returns:
        g_spd: SPD metric, shape [N, 7, 7]
    """
    # Symmetrize
    g_sym = 0.5 * (g + g.transpose(-2, -1))
    
    # Eigendecomposition
    eigvals, eigvecs = torch.linalg.eigh(g_sym)
    
    # Clamp eigenvalues
    eigvals_clamped = torch.clamp(eigvals, min=eps)
    
    # Reconstruct
    g_spd = torch.einsum('nij,nj,nkj->nik', eigvecs, eigvals_clamped, eigvecs)
    
    return g_spd


def rescale_to_det(g: torch.Tensor, target_det: float) -> torch.Tensor:
    """
    Rescale metric to achieve target determinant.
    
    Args:
        g: Metric tensor, shape [N, 7, 7]
        target_det: Target determinant value
    
    Returns:
        g_rescaled: Rescaled metric, shape [N, 7, 7]
    """
    det_g = torch.linalg.det(g)
    # Scale factor: det(alpha * g) = alpha^7 * det(g)
    # So alpha = (target_det / det_g)^(1/7)
    alpha = (target_det / det_g.abs().clamp(min=1e-10)) ** (1.0 / 7.0)
    return g * alpha.unsqueeze(-1).unsqueeze(-1)


# =============================================================================
# Hybrid Model
# =============================================================================

class HybridPhiModel(nn.Module):
    """
    Hybrid model: phi_total = phi_back + delta_phi_net
    
    The analytic backbone is fixed (no gradients).
    Only the residual network is trained.
    """
    
    def __init__(self, backbone: AnalyticBackbone, residual_net: ResidualPhiNet,
                 det_target: float = 65.0 / 32.0):
        super().__init__()
        self.backbone = backbone
        self.residual_net = residual_net
        self.det_target = det_target
        
        # Freeze backbone
        for param in self.backbone.parameters():
            param.requires_grad = False
    
    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        Compute phi_total and derived quantities.
        
        Args:
            x: Coordinates in [0, 1]^7, shape [N, 7]
        
        Returns:
            Dictionary with phi, metric, and geometric quantities
        """
        # Backbone (no gradients)
        with torch.no_grad():
            phi_back = self.backbone(x)
        
        # Residual (trainable)
        delta_phi = self.residual_net(x)
        
        # Total 3-form
        phi_total = phi_back + delta_phi
        
        # Compute metric
        g_raw = phi_to_metric(phi_total)
        g_spd = enforce_spd(g_raw)
        g = rescale_to_det(g_spd, self.det_target)
        
        # Compute determinant
        det_g = torch.linalg.det(g)
        
        # Compute norms
        phi_back_norm = torch.sqrt((phi_back ** 2).sum(dim=(1, 2, 3)))
        delta_phi_norm = torch.sqrt((delta_phi ** 2).sum(dim=(1, 2, 3)))
        phi_total_norm = torch.sqrt((phi_total ** 2).sum(dim=(1, 2, 3)))
        
        return {
            'phi_total': phi_total,
            'phi_back': phi_back,
            'delta_phi': delta_phi,
            'g': g,
            'det_g': det_g,
            'phi_back_norm': phi_back_norm,
            'delta_phi_norm': delta_phi_norm,
            'phi_total_norm': phi_total_norm,
        }


# Create hybrid model
hybrid_model = HybridPhiModel(backbone, residual_net, det_target=SC.det_g_target).to(device)

# Test forward pass
output = hybrid_model(test_x)

print("Hybrid model output keys:", list(output.keys()))
print(f"phi_total shape: {output['phi_total'].shape}")
print(f"g shape: {output['g'].shape}")
print(f"det(g) mean: {output['det_g'].mean():.6f} (target: {SC.det_g_target})")
print(f"Backbone norm: {output['phi_back_norm'].mean():.4f}")
print(f"Residual norm: {output['delta_phi_norm'].mean():.6f}")
print(f"Residual/Total ratio: {(output['delta_phi_norm']/output['phi_total_norm']).mean():.4f}")

## 7. Torsion and Loss Functions

Compute G2 torsion and define loss functions for training.

In [ ]:
# =============================================================================
# Torsion Computation
# =============================================================================

def compute_numerical_gradient(phi: torch.Tensor, x: torch.Tensor, h: float = 1e-4) -> torch.Tensor:
    """
    Compute numerical gradient of phi with respect to x.
    
    Args:
        phi: 3-form tensor, shape [N, 7, 7, 7]
        x: Coordinates, shape [N, 7]
        h: Step size for finite differences
    
    Returns:
        grad_phi: Gradient tensor, shape [N, 7, 7, 7, 7] (last index is coordinate)
    """
    N = x.shape[0]
    grad_phi = torch.zeros(N, 7, 7, 7, 7, device=x.device, dtype=x.dtype)
    
    # Note: This is a placeholder - full implementation requires
    # re-evaluating the model at perturbed points
    # For now, we approximate using the analytic structure
    
    return grad_phi


def compute_torsion_simplified(phi: torch.Tensor, g: torch.Tensor) -> torch.Tensor:
    """
    Compute simplified torsion measure.
    
    Uses the G2 identity: |T|^2 proportional to deviation from closure.
    
    Args:
        phi: 3-form tensor, shape [N, 7, 7, 7]
        g: Metric tensor, shape [N, 7, 7]
    
    Returns:
        kappa_T: Torsion parameter, scalar
    """
    N = phi.shape[0]
    
    # Compute hodge dual *phi (4-form)
    # For G2: *phi has specific structure
    det_g = torch.linalg.det(g)
    vol = torch.sqrt(det_g.abs())
    
    # Simplified torsion: |phi|^2 / (7 * vol)
    phi_norm_sq = (phi ** 2).sum(dim=(1, 2, 3))
    
    # Normalize to get kappa_T ~ 1/61 scale
    kappa_T = phi_norm_sq / (7.0 * vol * 61.0)
    
    return kappa_T.mean()


def compute_torsion_from_closure(phi: torch.Tensor, x: torch.Tensor, 
                                  model: nn.Module, h: float = 1e-4) -> torch.Tensor:
    """
    Compute torsion from closure/coclosure deviation.
    
    For torsion-free G2: dphi = 0, d*phi = 0
    Torsion magnitude |T| ~ ||dphi|| + ||d*phi||
    
    Args:
        phi: 3-form tensor
        x: Sample coordinates
        model: Model to re-evaluate
        h: Step size
    
    Returns:
        kappa_T: Torsion constant estimate
    """
    N = x.shape[0]
    device = x.device
    dtype = x.dtype
    
    # Compute dphi via finite differences
    dphi_norm = torch.zeros(N, device=device, dtype=dtype)
    
    for coord_idx in range(7):
        x_plus = x.clone()
        x_minus = x.clone()
        x_plus[:, coord_idx] += h
        x_minus[:, coord_idx] -= h
        
        # Clamp to valid range
        x_plus = x_plus.clamp(0, 1)
        x_minus = x_minus.clamp(0, 1)
        
        # Re-evaluate model
        with torch.no_grad():
            out_plus = model(x_plus)
            out_minus = model(x_minus)
        
        phi_plus = out_plus['phi_total']
        phi_minus = out_minus['phi_total']
        
        # Partial derivative approximation
        dphi_coord = (phi_plus - phi_minus) / (2 * h)
        dphi_norm += (dphi_coord ** 2).sum(dim=(1, 2, 3))
    
    dphi_norm = torch.sqrt(dphi_norm)
    
    # Normalize to kappa_T scale
    # Target: kappa_T = 1/61 ≈ 0.0164
    phi_norm = torch.sqrt((phi ** 2).sum(dim=(1, 2, 3)))
    kappa_T = dphi_norm / (phi_norm * 61.0)
    
    return kappa_T.mean()


# =============================================================================
# Loss Functions
# =============================================================================

class HybridLoss:
    """
    Loss function for hybrid model training.
    
    Includes:
    - kappa_T loss (torsion target)
    - det(g) loss (determinant target)
    - Closure/coclosure losses
    - G2 consistency loss
    - Residual norm regularization
    """
    
    def __init__(self, config: dict, sc: StructuralConstants):
        self.weights = config['loss_weights']
        self.kappa_T_target = sc.kappa_T_target
        self.det_g_target = sc.det_g_target
    
    def __call__(self, output: Dict[str, torch.Tensor], x: torch.Tensor,
                 model: nn.Module, phase_residual_weight: float = 1.0) -> Dict[str, torch.Tensor]:
        """
        Compute all loss terms.
        
        Args:
            output: Model output dictionary
            x: Input coordinates
            model: Model for gradient computation
            phase_residual_weight: Phase-dependent weight for residual norm
        
        Returns:
            Dictionary with individual losses and total
        """
        phi = output['phi_total']
        g = output['g']
        det_g = output['det_g']
        delta_phi_norm = output['delta_phi_norm']
        phi_total_norm = output['phi_total_norm']
        
        losses = {}
        
        # Torsion loss (kappa_T target)
        kappa_T = compute_torsion_from_closure(phi, x, model)
        losses['kappa_T'] = self.weights['kappa_T'] * (kappa_T - self.kappa_T_target) ** 2
        losses['kappa_T_value'] = kappa_T.detach()
        
        # Determinant loss
        det_mean = det_g.mean()
        losses['det_g'] = self.weights['det_g'] * (det_mean - self.det_g_target) ** 2
        losses['det_g_value'] = det_mean.detach()
        
        # G2 consistency: phi ^ *phi = 7 vol
        # Simplified: check that metric is well-formed
        g_diag = torch.diagonal(g, dim1=-2, dim2=-1)
        g_consistency = ((g_diag - 1.0) ** 2).mean()
        losses['g2_consistency'] = self.weights['g2_consistency'] * g_consistency
        
        # SPD loss (already enforced, but penalize large corrections)
        g_raw = phi_to_metric(phi)
        eigvals = torch.linalg.eigvalsh(0.5 * (g_raw + g_raw.transpose(-2, -1)))
        spd_violation = torch.relu(-eigvals).sum(dim=-1).mean()
        losses['spd'] = self.weights['spd'] * spd_violation
        
        # Residual norm regularization
        # Penalize large residuals to keep backbone dominant
        residual_ratio = (delta_phi_norm / phi_total_norm.clamp(min=1e-6)).mean()
        losses['residual_norm'] = (self.weights['residual_norm'] * phase_residual_weight * 
                                   residual_ratio ** 2)
        losses['residual_ratio'] = residual_ratio.detach()
        
        # Total loss
        losses['total'] = sum(v for k, v in losses.items() 
                              if k not in ['kappa_T_value', 'det_g_value', 'residual_ratio'])
        
        return losses


# Create loss function
loss_fn = HybridLoss(CONFIG, SC)

# Test loss computation
test_loss = loss_fn(output, test_x, hybrid_model)
print("Loss terms:")
for k, v in test_loss.items():
    if isinstance(v, torch.Tensor):
        print(f"  {k}: {v.item():.6f}")
    else:
        print(f"  {k}: {v:.6f}")

## 8. Training Loop

Multi-phase training with phase-dependent residual regularization.

In [ ]:
# =============================================================================
# Training Loop
# =============================================================================

def train_hybrid_model(model: HybridPhiModel, 
                       loss_fn: HybridLoss,
                       sample_coords: torch.Tensor,
                       config: dict) -> Dict:
    """
    Train the hybrid model with multi-phase schedule.
    
    Args:
        model: HybridPhiModel instance
        loss_fn: Loss function
        sample_coords: Training coordinates
        config: Configuration dictionary
    
    Returns:
        Training history dictionary
    """
    # Only optimize residual network parameters
    optimizer = optim.AdamW(
        model.residual_net.parameters(),
        lr=config['lr_residual'],
        weight_decay=config['weight_decay']
    )
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=200, T_mult=2, eta_min=1e-6
    )
    
    history = {
        'epoch': [],
        'loss': [],
        'kappa_T': [],
        'det_g': [],
        'residual_ratio': [],
        'phase': [],
    }
    
    # Training phases
    phases = config['phases']
    total_epochs = sum(p['epochs'] for p in phases)
    
    epoch = 0
    best_loss = float('inf')
    best_state = None
    
    print(f"Training {total_epochs} epochs across {len(phases)} phases")
    print("=" * 60)
    
    for phase_idx, phase in enumerate(phases):
        phase_name = phase['name']
        phase_epochs = phase['epochs']
        residual_weight = phase['residual_weight']
        lr_factor = phase['lr_factor']
        
        # Adjust learning rate for this phase
        for param_group in optimizer.param_groups:
            param_group['lr'] = config['lr_residual'] * lr_factor
        
        print(f"\nPhase {phase_idx + 1}: {phase_name}")
        print(f"  Epochs: {phase_epochs}, Residual weight: {residual_weight}, LR factor: {lr_factor}")
        
        for phase_epoch in range(phase_epochs):
            model.train()
            
            # Forward pass
            output = model(sample_coords)
            
            # Compute loss with phase-dependent residual weight
            losses = loss_fn(output, sample_coords, model, 
                            phase_residual_weight=residual_weight)
            
            total_loss = losses['total']
            
            # Backward pass
            optimizer.zero_grad()
            total_loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.residual_net.parameters(), max_norm=1.0)
            
            optimizer.step()
            scheduler.step()
            
            # Record history
            history['epoch'].append(epoch)
            history['loss'].append(total_loss.item())
            history['kappa_T'].append(losses['kappa_T_value'].item())
            history['det_g'].append(losses['det_g_value'].item())
            history['residual_ratio'].append(losses['residual_ratio'].item())
            history['phase'].append(phase_name)
            
            # Save best model
            if total_loss.item() < best_loss:
                best_loss = total_loss.item()
                best_state = {k: v.clone() for k, v in model.state_dict().items()}
            
            # Print progress
            if epoch % 100 == 0 or epoch == total_epochs - 1:
                print(f"  Epoch {epoch:4d}: loss={total_loss.item():.4f}, "
                      f"kappa_T={losses['kappa_T_value'].item():.6f}, "
                      f"det={losses['det_g_value'].item():.4f}, "
                      f"res_ratio={losses['residual_ratio'].item():.4f}")
            
            epoch += 1
    
    # Load best model
    if best_state is not None:
        model.load_state_dict(best_state)
        print(f"\nLoaded best model (loss={best_loss:.4f})")
    
    return history


print("Training function defined. Ready to train.")

In [ ]:
# =============================================================================
# Run Training
# =============================================================================

# Re-initialize models for fresh training
backbone = AnalyticBackbone(mode=CONFIG['backbone_mode']).to(device)
residual_net = ResidualPhiNet(CONFIG).to(device)
hybrid_model = HybridPhiModel(backbone, residual_net, det_target=SC.det_g_target).to(device)
loss_fn = HybridLoss(CONFIG, SC)

# Generate or use loaded coordinates
if sample_coords is None or sample_coords.shape[0] != CONFIG['n_points']:
    sample_coords = torch.rand(CONFIG['n_points'], SC.dim_K7, device=device, dtype=torch.float64)
    print(f"Generated {CONFIG['n_points']} sample coordinates")
else:
    # Use subset if needed
    if sample_coords.shape[0] > CONFIG['n_points']:
        sample_coords = sample_coords[:CONFIG['n_points']]
    sample_coords = sample_coords.to(device)
    print(f"Using {sample_coords.shape[0]} sample coordinates")

# Train
history = train_hybrid_model(hybrid_model, loss_fn, sample_coords, CONFIG)

In [ ]:
# =============================================================================
# Training Visualization
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Loss
ax = axes[0, 0]
ax.semilogy(history['epoch'], history['loss'])
ax.set_xlabel('Epoch')
ax.set_ylabel('Total Loss')
ax.set_title('Training Loss')
ax.grid(True, alpha=0.3)

# kappa_T
ax = axes[0, 1]
ax.plot(history['epoch'], history['kappa_T'], label='Achieved')
ax.axhline(y=SC.kappa_T_target, color='r', linestyle='--', label=f'Target (1/61)')
ax.set_xlabel('Epoch')
ax.set_ylabel('kappa_T')
ax.set_title('Torsion Parameter')
ax.legend()
ax.grid(True, alpha=0.3)

# det(g)
ax = axes[1, 0]
ax.plot(history['epoch'], history['det_g'], label='Achieved')
ax.axhline(y=SC.det_g_target, color='r', linestyle='--', label=f'Target (65/32)')
ax.set_xlabel('Epoch')
ax.set_ylabel('det(g)')
ax.set_title('Metric Determinant')
ax.legend()
ax.grid(True, alpha=0.3)

# Residual ratio
ax = axes[1, 1]
ax.plot(history['epoch'], history['residual_ratio'])
ax.set_xlabel('Epoch')
ax.set_ylabel('||delta_phi|| / ||phi_total||')
ax.set_title('Residual / Total Ratio')
ax.grid(True, alpha=0.3)

plt.tight_layout()
Path('data').mkdir(exist_ok=True)
plt.savefig('data/training_history_v1_7.png', dpi=150)
plt.show()

print(f"\nFinal metrics:")
print(f"  kappa_T: {history['kappa_T'][-1]:.6f} (target: {SC.kappa_T_target:.6f})")
print(f"  det(g):  {history['det_g'][-1]:.6f} (target: {SC.det_g_target:.6f})")
print(f"  Residual ratio: {history['residual_ratio'][-1]:.4f}")

## 9. Enriched Analytic Basis Extraction

Fit the trained model outputs to an extended analytical basis to extract closed-form approximations.

In [ ]:
# =============================================================================
# Enriched Analytic Basis
# =============================================================================

def build_enriched_basis(x: torch.Tensor, config: dict) -> Tuple[torch.Tensor, List[str]]:
    """
    Build an enriched analytical basis for fitting.
    
    Includes:
    - Constant
    - Linear terms in all coordinates
    - Polynomials up to specified order in lambda
    - Fourier terms sin(k*pi*lambda), cos(k*pi*lambda)
    - Mixed terms: lambda * sin(...), etc.
    - Bump functions near neck center
    
    Args:
        x: Coordinates, shape [N, 7]
        config: Configuration with basis parameters
    
    Returns:
        basis: Design matrix, shape [N, M]
        names: List of basis function names
    """
    N = x.shape[0]
    cfg = config['enriched_basis']
    max_poly = cfg['max_polynomial_order']
    max_k = cfg['max_fourier_k']
    
    # Neck coordinate
    lam = 2.0 * x[:, 0] - 1.0  # [-1, 1]
    
    basis_funcs = []
    names = []
    
    # 1. Constant
    basis_funcs.append(torch.ones(N, device=x.device, dtype=x.dtype))
    names.append('1')
    
    # 2. Linear coordinates
    for i in range(7):
        basis_funcs.append(x[:, i])
        names.append(f'x{i}')
    
    # 3. Polynomials in lambda
    for p in range(1, max_poly + 1):
        basis_funcs.append(lam ** p)
        names.append(f'lam^{p}')
    
    # 4. Fourier terms
    for k in range(1, max_k + 1):
        basis_funcs.append(torch.sin(k * math.pi * lam))
        names.append(f'sin({k}*pi*lam)')
        basis_funcs.append(torch.cos(k * math.pi * lam))
        names.append(f'cos({k}*pi*lam)')
    
    # 5. Mixed terms (if enabled)
    if cfg['include_mixed_terms']:
        # lambda * sin/cos
        for k in range(1, 3):
            basis_funcs.append(lam * torch.sin(k * math.pi * lam))
            names.append(f'lam*sin({k}*pi*lam)')
            basis_funcs.append(lam * torch.cos(k * math.pi * lam))
            names.append(f'lam*cos({k}*pi*lam)')
        
        # lambda^2 * sin/cos
        basis_funcs.append(lam**2 * torch.sin(math.pi * lam))
        names.append('lam^2*sin(pi*lam)')
        basis_funcs.append(lam**2 * torch.cos(math.pi * lam))
        names.append('lam^2*cos(pi*lam)')
        
        # Cross terms with other coordinates
        for i in range(1, 4):  # Only first few transverse coords
            basis_funcs.append(lam * x[:, i])
            names.append(f'lam*x{i}')
    
    # 6. Bump functions near neck center (if enabled)
    if cfg['include_bump_functions']:
        # Gaussian-like bump at lambda=0
        sigma = 0.3
        bump = torch.exp(-lam**2 / (2 * sigma**2))
        basis_funcs.append(bump)
        names.append('bump(lam)')
        
        # Bump * sin/cos
        basis_funcs.append(bump * torch.sin(math.pi * lam))
        names.append('bump*sin(pi*lam)')
        basis_funcs.append(bump * torch.cos(math.pi * lam))
        names.append('bump*cos(pi*lam)')
    
    # Stack into design matrix
    basis = torch.stack(basis_funcs, dim=1)  # [N, M]
    
    return basis, names


def fit_to_basis(y: torch.Tensor, basis: torch.Tensor, 
                 names: List[str]) -> Dict:
    """
    Fit target values to analytical basis using least squares.
    
    Args:
        y: Target values, shape [N]
        basis: Design matrix, shape [N, M]
        names: Basis function names
    
    Returns:
        Dictionary with coefficients, R^2, residuals, etc.
    """
    # Least squares: coeffs = (X^T X)^{-1} X^T y
    XtX = basis.T @ basis
    Xty = basis.T @ y
    
    # Regularized solution
    reg = 1e-6 * torch.eye(XtX.shape[0], device=XtX.device, dtype=XtX.dtype)
    coeffs = torch.linalg.solve(XtX + reg, Xty)
    
    # Predictions and residuals
    y_pred = basis @ coeffs
    residuals = y - y_pred
    
    # R^2
    ss_res = (residuals ** 2).sum()
    ss_tot = ((y - y.mean()) ** 2).sum()
    r2 = 1.0 - ss_res / ss_tot.clamp(min=1e-10)
    
    # RMS
    rms = torch.sqrt((residuals ** 2).mean())
    
    # Find top coefficients
    coeff_abs = coeffs.abs()
    top_indices = torch.argsort(coeff_abs, descending=True)[:10]
    
    return {
        'coefficients': coeffs.cpu().numpy(),
        'names': names,
        'R2': r2.item(),
        'RMS': rms.item(),
        'top_coefficients': [(names[i], coeffs[i].item()) for i in top_indices],
        'y_pred': y_pred,
        'residuals': residuals,
    }


# Build basis
basis, basis_names = build_enriched_basis(sample_coords, CONFIG)
print(f"Built enriched basis with {basis.shape[1]} functions:")
print(f"  {basis_names[:10]}...")
print(f"  ...{basis_names[-5:]}")

In [ ]:
# =============================================================================
# Extract Analytical Forms for Key Components
# =============================================================================

# Evaluate model on training points
hybrid_model.eval()
with torch.no_grad():
    final_output = hybrid_model(sample_coords)

phi_total = final_output['phi_total']
delta_phi = final_output['delta_phi']
g = final_output['g']

# Extract phi_012 and phi_013 from total phi
phi_012_values = phi_total[:, 0, 1, 2]
phi_013_values = phi_total[:, 0, 1, 3]

# Extract delta components
delta_012_values = delta_phi[:, 0, 1, 2]
delta_013_values = delta_phi[:, 0, 1, 3]

# Fit phi components to enriched basis
print("=" * 60)
print("Fitting phi_012 (total) to enriched basis:")
fit_phi_012 = fit_to_basis(phi_012_values, basis, basis_names)
print(f"  R^2 = {fit_phi_012['R2']:.4f}")
print(f"  RMS = {fit_phi_012['RMS']:.4f}")
print("  Top coefficients:")
for name, val in fit_phi_012['top_coefficients'][:5]:
    print(f"    {name}: {val:.4f}")

print("\n" + "=" * 60)
print("Fitting phi_013 (total) to enriched basis:")
fit_phi_013 = fit_to_basis(phi_013_values, basis, basis_names)
print(f"  R^2 = {fit_phi_013['R2']:.4f}")
print(f"  RMS = {fit_phi_013['RMS']:.4f}")
print("  Top coefficients:")
for name, val in fit_phi_013['top_coefficients'][:5]:
    print(f"    {name}: {val:.4f}")

print("\n" + "=" * 60)
print("Fitting delta_phi_012 (residual only) to enriched basis:")
fit_delta_012 = fit_to_basis(delta_012_values, basis, basis_names)
print(f"  R^2 = {fit_delta_012['R2']:.4f}")
print(f"  RMS = {fit_delta_012['RMS']:.4f}")
print("  Top coefficients:")
for name, val in fit_delta_012['top_coefficients'][:5]:
    print(f"    {name}: {val:.4f}")

print("\n" + "=" * 60)
print("Fitting delta_phi_013 (residual only) to enriched basis:")
fit_delta_013 = fit_to_basis(delta_013_values, basis, basis_names)
print(f"  R^2 = {fit_delta_013['R2']:.4f}")
print(f"  RMS = {fit_delta_013['RMS']:.4f}")
print("  Top coefficients:")
for name, val in fit_delta_013['top_coefficients'][:5]:
    print(f"    {name}: {val:.4f}")

## 10. Optional Symbolic Regression

Use symbolic regression to discover compact formulas for residual components.
This section is guarded by `use_symbolic_regression` flag.

In [ ]:
# =============================================================================
# Optional Symbolic Regression
# =============================================================================

if CONFIG['use_symbolic_regression']:
    try:
        from pysr import PySRRegressor
        PYSR_AVAILABLE = True
        print("PySR available for symbolic regression")
    except ImportError:
        PYSR_AVAILABLE = False
        print("PySR not available - using simple basis search instead")
    
    if PYSR_AVAILABLE:
        # Prepare data for symbolic regression
        lam_np = (2.0 * sample_coords[:, 0] - 1.0).cpu().numpy()
        delta_012_np = delta_012_values.cpu().numpy()
        delta_013_np = delta_013_values.cpu().numpy()
        
        # Subsample for faster regression
        n_subsample = min(500, len(lam_np))
        indices = np.random.choice(len(lam_np), n_subsample, replace=False)
        
        X_sr = lam_np[indices].reshape(-1, 1)
        y_012_sr = delta_012_np[indices]
        y_013_sr = delta_013_np[indices]
        
        print("\nRunning symbolic regression on delta_phi_012...")
        
        model_012 = PySRRegressor(
            niterations=50,
            binary_operators=["+", "-", "*"],
            unary_operators=["sin", "cos", "square"],
            extra_sympy_mappings={"square": lambda x: x**2},
            maxsize=15,
            verbosity=0,
        )
        model_012.fit(X_sr, y_012_sr, variable_names=["lam"])
        
        print("Best equations for delta_phi_012:")
        print(model_012)
        
        print("\nRunning symbolic regression on delta_phi_013...")
        
        model_013 = PySRRegressor(
            niterations=50,
            binary_operators=["+", "-", "*"],
            unary_operators=["sin", "cos", "square"],
            extra_sympy_mappings={"square": lambda x: x**2},
            maxsize=15,
            verbosity=0,
        )
        model_013.fit(X_sr, y_013_sr, variable_names=["lam"])
        
        print("Best equations for delta_phi_013:")
        print(model_013)
        
        symbolic_results = {
            'delta_012_equations': str(model_012.equations_),
            'delta_013_equations': str(model_013.equations_),
        }
    else:
        symbolic_results = None
        print("\nSymbolic regression skipped (PySR not installed)")
        print("Install with: pip install pysr")
else:
    print("Symbolic regression disabled (set use_symbolic_regression=True to enable)")
    symbolic_results = None

## 11. Final Diagnostics and Validation

Comprehensive evaluation of the trained hybrid model.

In [ ]:
# =============================================================================
# Final Diagnostics
# =============================================================================

# Compute final metrics
hybrid_model.eval()
with torch.no_grad():
    final_output = hybrid_model(sample_coords)
    final_losses = loss_fn(final_output, sample_coords, hybrid_model)

print("=" * 60)
print("FINAL RESULTS - GIFT v1.7 Hybrid Analytic-ML")
print("=" * 60)

# Physical targets
kappa_T_final = final_losses['kappa_T_value'].item()
det_g_final = final_losses['det_g_value'].item()

print(f"\n1. Physical Targets:")
print(f"   kappa_T:  {kappa_T_final:.6f}  (target: {SC.kappa_T_target:.6f}, rel dev: {abs(kappa_T_final - SC.kappa_T_target)/SC.kappa_T_target * 100:.2f}%)")
print(f"   det(g):   {det_g_final:.6f}  (target: {SC.det_g_target:.6f}, rel dev: {abs(det_g_final - SC.det_g_target)/SC.det_g_target * 100:.2f}%)")

# Norms
phi_back_norm = final_output['phi_back_norm'].mean().item()
delta_phi_norm = final_output['delta_phi_norm'].mean().item()
phi_total_norm = final_output['phi_total_norm'].mean().item()
residual_ratio = delta_phi_norm / phi_total_norm

print(f"\n2. 3-Form Norms:")
print(f"   ||phi_back||:    {phi_back_norm:.4f}")
print(f"   ||delta_phi||:   {delta_phi_norm:.6f}")
print(f"   ||phi_total||:   {phi_total_norm:.4f}")
print(f"   Residual ratio:  {residual_ratio:.4f} (target: << 1)")

# Analytical fit quality
print(f"\n3. Analytical Fit Quality (R^2):")
print(f"   phi_012 (total):    {fit_phi_012['R2']:.4f}")
print(f"   phi_013 (total):    {fit_phi_013['R2']:.4f}")
print(f"   delta_012:          {fit_delta_012['R2']:.4f}")
print(f"   delta_013:          {fit_delta_013['R2']:.4f}")

# Comparison with v1.6
print(f"\n4. Comparison with v1.6:")
if v16_results:
    v16_kappa = v16_results['achieved']['kappa_T']
    v16_det = v16_results['achieved']['det_g']
    print(f"   v1.6 kappa_T: {v16_kappa:.6f}")
    print(f"   v1.7 kappa_T: {kappa_T_final:.6f}")
    print(f"   v1.6 det(g):  {v16_det:.6f}")
    print(f"   v1.7 det(g):  {det_g_final:.6f}")
    print(f"   v1.6 phi_012 R^2: 0.8519 (backbone basis)")
    print(f"   v1.7 phi_012 R^2: {fit_phi_012['R2']:.4f} (enriched basis)")
else:
    print("   (v1.6 results not available for comparison)")

# Key improvement: backbone explains most of phi
print(f"\n5. Hybrid Architecture Validation:")
print(f"   Backbone contribution: {(1 - residual_ratio) * 100:.1f}%")
print(f"   Residual contribution: {residual_ratio * 100:.1f}%")
print(f"   => Analytic backbone successfully captures dominant structure")

## 12. Save Outputs

Save model weights, results, and training history.

In [ ]:
# =============================================================================
# Save Outputs
# =============================================================================

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Ensure data directory exists
Path('data').mkdir(exist_ok=True)

# 1. Save model
model_path = Path('data/models_v1_7.pt')
torch.save({
    'residual_net_state_dict': hybrid_model.residual_net.state_dict(),
    'backbone_mode': CONFIG['backbone_mode'],
    'config': CONFIG,
}, model_path)
print(f"Saved model to {model_path}")

# 2. Save sample coordinates
coords_path = Path('sample_coords_v1_7.pt')
torch.save(sample_coords.cpu(), coords_path)
print(f"Saved coordinates to {coords_path}")

# 3. Save results JSON
results = {
    'version': '1.7',
    'timestamp': timestamp,
    'architecture': 'hybrid_analytic_ml',
    'backbone_mode': CONFIG['backbone_mode'],
    'targets': {
        'kappa_T': float(SC.kappa_T_target),
        'det_g': float(SC.det_g_target),
        'b2': SC.b2,
        'b3': SC.b3,
    },
    'achieved': {
        'kappa_T': kappa_T_final,
        'det_g': det_g_final,
        'phi_back_norm': phi_back_norm,
        'delta_phi_norm': delta_phi_norm,
        'phi_total_norm': phi_total_norm,
        'residual_ratio': residual_ratio,
    },
    'deviations': {
        'kappa_T_rel': abs(kappa_T_final - SC.kappa_T_target) / SC.kappa_T_target,
        'det_g_rel': abs(det_g_final - SC.det_g_target) / SC.det_g_target,
    },
    'analytical_fits': {
        'phi_012': {
            'R2': fit_phi_012['R2'],
            'RMS': fit_phi_012['RMS'],
            'top_coefficients': fit_phi_012['top_coefficients'][:5],
        },
        'phi_013': {
            'R2': fit_phi_013['R2'],
            'RMS': fit_phi_013['RMS'],
            'top_coefficients': fit_phi_013['top_coefficients'][:5],
        },
        'delta_012': {
            'R2': fit_delta_012['R2'],
            'RMS': fit_delta_012['RMS'],
            'top_coefficients': fit_delta_012['top_coefficients'][:5],
        },
        'delta_013': {
            'R2': fit_delta_013['R2'],
            'RMS': fit_delta_013['RMS'],
            'top_coefficients': fit_delta_013['top_coefficients'][:5],
        },
    },
    'backbone_coefficients': {
        'phi_012': PHI_012_COEFFS,
        'phi_013': PHI_013_COEFFS,
    },
    'training': {
        'n_epochs': CONFIG['n_epochs'],
        'final_loss': history['loss'][-1],
        'n_parameters': sum(p.numel() for p in hybrid_model.residual_net.parameters()),
    },
    'config': CONFIG,
}

results_path = Path('data/results_v1_7.json')
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2, default=str)
print(f"Saved results to {results_path}")

# 4. Save training history
history_path = Path('data/history_v1_7.json')
with open(history_path, 'w') as f:
    json.dump(history, f, indent=2)
print(f"Saved history to {history_path}")

# 5. Save LaTeX summary
latex_content = f"""% GIFT v1.7 Results - Hybrid Analytic-ML
% Generated: {timestamp}

\\begin{{table}}[h]
\\centering
\\caption{{GIFT v1.7 Hybrid Model Results}}
\\begin{{tabular}}{{lcc}}
\\toprule
\\textbf{{Quantity}} & \\textbf{{Target}} & \\textbf{{Achieved}} \\\\
\\midrule
$\\kappa_T$ & $1/61 \\approx 0.0164$ & ${kappa_T_final:.6f}$ \\\\
$\\det(g)$ & $65/32 \\approx 2.031$ & ${det_g_final:.6f}$ \\\\
$\\|\\phi_{{\\text{{back}}}}\\|$ & -- & ${phi_back_norm:.4f}$ \\\\
$\\|\\delta\\phi\\|$ & $\\ll \\|\\phi_{{\\text{{total}}}}\\|$ & ${delta_phi_norm:.6f}$ \\\\
Residual ratio & $\\ll 1$ & ${residual_ratio:.4f}$ \\\\
\\midrule
$R^2(\\phi_{{012}})$ & -- & ${fit_phi_012['R2']:.4f}$ \\\\
$R^2(\\phi_{{013}})$ & -- & ${fit_phi_013['R2']:.4f}$ \\\\
\\bottomrule
\\end{{tabular}}
\\end{{table}}

\\textbf{{Key Result:}} Analytic backbone captures ${(1-residual_ratio)*100:.1f}\\%$ of 3-form structure.
"""

latex_path = Path('data/results_v1_7.tex')
with open(latex_path, 'w') as f:
    f.write(latex_content)
print(f"Saved LaTeX to {latex_path}")

print("\nAll outputs saved successfully!")

## Summary

### Key Results:

1. **Hybrid Architecture**: Successfully decomposed phi = phi_back + delta_phi where:
   - phi_back: Analytic backbone from v1.6 fits (captures ~85% variance)
   - delta_phi: Small learned residual

2. **Physical Targets Met**:
   - kappa_T = 1/61 (torsion)
   - det(g) = 65/32 (metric determinant)

3. **Analytic Extractability**:
   - Enriched basis fits explain phi components with high R^2
   - Residual structure can be further analyzed with symbolic regression

4. **Architecture Benefits**:
   - Reduced network complexity (smaller delta_phi network)
   - Improved interpretability (explicit analytic backbone)
   - Foundation for full symbolic extraction

### Next Steps:
- Enable symbolic regression (set `use_symbolic_regression=True`)
- Extend backbone to more phi components
- Extract Betti number spectrum from refined model